In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import re
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.datasets import fetch_20newsgroups, load_files

import pandas as pd
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

from tqdm.auto import tqdm

# df_orig = pd.read_csv('./data/data.csv')

In [2]:
geolocator = Nominatim(user_agent="my_geocoder")

def replace_based_on_pattern(value, pattern, new_value):
    if re.search(pattern, value):
        return new_value
    return value


def convert(cell_val):
    """
    Purpose: 
    """
    pattern = r"\d+(,\d+)?(\.\d+)?"
    num_str = re.search(pattern, cell_val)

    if (num_str):
        # comment: 
        return float(num_str.group().replace(',' , ''))
    else:
        return None
        # comment: 
    # end if
# end def

def addr_to_coords(addr, geolocator):
    """
    Purpose: 
    """
    geolocator = Nominatim(user_agent="my_geocoder", scheme='http', domain='localhost:8080', timeout=10)

    # Геокодирование адреса
    try:
        # comment: 
        location = geolocator.geocode(addr)
        if (location):
        # comment:
            return (location.latitude, location.longitude)
        else:
            # comment: 
            return None
    # end if
    except (GeocoderTimedOut, GeocoderUnavailable) as e:
        print(f"Error: {e}. Retrying...")
        time.sleep(2)  # Добавляем задержку перед повторной попыткой
        return addr_to_coords(addr, geolocator)
    # end try

def convert_bath(cell_val):
    """
    Purpose: 
    """
    pattern = r"\d+[,.]?\d*"
    num_str = re.search(pattern, cell_val)

    if (num_str):
        # comment: 
        return float(num_str.group().replace(',' , ''))
    else:
        return None
        # comment: 
    # end if
# end def

def value_percentage(df, column):
    """
    Purpose: процент значений колонки от общего числа строк
    """
    name_series = df[column].value_counts(dropna=False)
    name_df = name_series.to_frame()
    name_df.reset_index(inplace=True)
    name_df['percent'] = name_df['count'] / df.shape[0] * 100
    return name_df

# end def

# Пользовательская функция для объединения столбцов по условию
def combine_columns(row):
    if ((row['private_pool'] + row['privatepool']) < 1):
        return 0
    else:
        return 1
    
def missing_values_per_column(df):
    """
    Purpose: вывод количества отсутствующих значений в датасете
    """
    missing_values = df.isna().sum()
    print("Количество значений NaN по каждой колонке:")
    print(missing_values)

# end def

df = df_orig.copy()

df.drop(['zipcode', 'mls-id', 'MlsId'], axis=1, inplace=True)

df['street'] = df['street'].astype(str).str.strip()
df['state'] = df['state'].astype(str).str.strip()
df['full_addr'] = df['street'] +', '+ df['state']

tqdm.pandas(desc="Processing rows")

df100 = df.copy()

df100['coords'] = df100['full_addr'].progress_apply(lambda x: addr_to_coords(x, geolocator))
df100[['latitude', 'longitude']] = df100['coords'].apply(lambda x: pd.Series(x) if x is not None else pd.Series([None, None]))

df100.to_csv('./data/data_with_coords50000.csv', mode='a+', header=False)

In [3]:
df = pd.read_csv('data/data_with_coords50000.csv', index_col=0)
df.columns = map(str.lower, df.columns)
df.columns = df.columns.str.replace(' ', '_')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 377185 entries, 0 to 377184
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   status        337267 non-null  object 
 1   private_pool  4181 non-null    object 
 2   propertytype  342452 non-null  object 
 3   street        377183 non-null  object 
 4   baths         270847 non-null  object 
 5   homefacts     377185 non-null  object 
 6   fireplace     103114 non-null  object 
 7   city          377151 non-null  object 
 8   schools       377185 non-null  object 
 9   sqft          336608 non-null  object 
 10  beds          285903 non-null  object 
 11  state         377185 non-null  object 
 12  stories       226469 non-null  object 
 13  privatepool   40311 non-null   object 
 14  target        374704 non-null  object 
 15  full_addr     377185 non-null  object 
 16  coords        264625 non-null  object 
 17  latitude      264625 non-null  float64
 18  longitude

Изменим имена колонок

In [5]:
df = df.rename(columns={
    'propertytype': 'property_type',
    'latitude': 'lat',
    'longitude': 'lon'
})

Удалим колонки, участвовавшие в геокодировании

In [6]:
df = df.dropna(subset=['coords'])
labels = [
    'street',
    'city',
    'full_addr',
    'coords'
]
df.drop(labels=labels, axis=1, inplace=True)


In [7]:
missing_values_per_column(df)

Количество значений NaN по каждой колонке:
status            25067
private_pool     261609
property_type     20641
baths             71332
homefacts             0
fireplace        177950
schools               0
sqft              27122
beds              58205
state                 0
stories           97523
privatepool      239905
target             2103
lat                   0
lon                   0
dtype: int64


точно можно удалить строки с нулевыми значениями в target, толку от неизвестной цены не будет

In [8]:
df = df.dropna(subset=['target'])

df['target'] = df['target'].astype(str)
df['target'] = df['target'].map(convert)

Обработаем столбец 'private_pool'

In [9]:
df.loc[df['private_pool'] == 'Yes', 'private_pool'] = 1
df['private_pool'].fillna(0, inplace=True)
df['private_pool'] = df['private_pool'].astype(int)

Обработаем столбец 'privatepool'

In [10]:
df.loc[df['privatepool'] == 'Yes', 'privatepool'] = 1
df.loc[df['privatepool'] == 'yes', 'privatepool'] = 1
df['privatepool'].fillna(0, inplace=True)
df['privatepool'] = df['privatepool'].astype(int)

Объединим столбцы 'private_pool' и 'privatepool'

In [11]:
df['pool'] = df.apply(combine_columns, axis=1)

In [12]:
df.drop(['private_pool', 'privatepool'], axis=1, inplace=True)

In [13]:
df.sample(5,random_state=42)

,status,property_type,baths,homefacts,fireplace,schools,sqft,beds,state,stories,target,lat,lon,pool
37869,Active,Land,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,"[{'rating': ['2', '1', '3', '3', '6', 'NR', 'N...",0,NaN,FL,NaN,27500.0,30.185343,-84.210476,0
200607,for sale,multi-family,2 Baths,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",NaN,"[{'rating': ['3/10', '2/10', '3/10'], 'data': ...","1,280 sqft",3 Beds,PA,2.0,225000.0,40.051991,-75.074784,0
20987,For sale,Multi Family,NaN,"{'atAGlanceFacts': [{'factValue': '1930', 'fac...",NaN,"[{'rating': ['1/10', '2/10', '1/10'], 'data': ...","1,021 sqft",0,IN,0,65000.0,39.774322,-86.112537,0
44099,For sale,Single Family,Bathrooms: 8,"{'atAGlanceFacts': [{'factValue': '2004', 'fac...",NaN,"[{'rating': ['10/10', '10/10', '9/10'], 'data'...","Total interior livable area: 5,641 sqft",5 bd,TX,2.0,1890.0,30.189525,-95.543038,1
261359,for sale,single-family home,3 Baths,"{'atAGlanceFacts': [{'factValue': '1938', 'fac...",yes,"[{'rating': ['7/10', '7/10', '6/10'], 'data': ...","2,146 sqft",4 Beds,TX,1.0,619900.0,29.481529,-98.471703,1


Приведем статус for sale к единому виду

In [14]:
df.loc[df['status'] == 'For sale', 'status'] = 'for_sale'
df.loc[df['status'] == 'for sale', 'status'] = 'for_sale'

Посмотрим, какие значения есть в колонке status и как они распределяются

In [15]:
status_df = value_percentage(df, 'status')
status_df

,status,count,percent
0,for_sale,139938,53.305247
1,Active,74157,28.247918
2,NaN,24558,9.354645
3,foreclosure,5059,1.927077
4,Pending,3974,1.513778
...,...,...,...
144,Coming soon: Oct 29.,1,0.000381
145,Pending W/Backup Wanted,1,0.000381
146,Pending Backups Wanted,1,0.000381
147,Pending Bring Backup,1,0.000381


Оставим объекты, которые в данный момент активны и продаются

In [16]:
filtered_status_df = status_df.loc[status_df['percent'] >= 2]
filtered_status_df

,status,count,percent
0,for_sale,139938,53.305247
1,Active,74157,28.247918
2,NaN,24558,9.354645


In [17]:
status_list = filtered_status_df['status'].to_list()
status_list

['for_sale', 'Active', nan]

In [18]:
df = df.loc[df['status'].isin(status_list)]

In [19]:
df.sample(5)

,status,property_type,baths,homefacts,fireplace,schools,sqft,beds,state,stories,target,lat,lon,pool
255451,for_sale,single-family home,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,"[{'rating': ['1/10', 'None/10', '2/10', '1/10'...","2,125 sqft",4 Beds,TX,2.0,245278.0,31.688092,-96.492902,0
184699,for_sale,multi-family,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,"[{'rating': ['1/10', '4/10', 'None/10', '5/10'...","1,358 sqft",3 Beds,NY,2,135000.0,42.846126,-78.825650,0
6415,for_sale,lot/land,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,"[{'rating': ['9/10', '7/10', '4/10'], 'data': ...",NaN,NaN,UT,NaN,459900.0,40.648004,-111.806102,0
217761,Active,"1 Story, Manufactured Home - Double Wide",NaN,"{'atAGlanceFacts': [{'factValue': '1996', 'fac...",NaN,"[{'rating': ['3', 'NR', '3', '1', '3'], 'data'...","1,216",NaN,TX,NaN,130000.0,29.171535,-98.456062,0
30230,for_sale,lot/land,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,"[{'rating': ['8/10', '6/10', '7/10'], 'data': ...","14,262 sqft",NaN,TX,NaN,70000.0,29.553424,-98.475402,0


Закодируем статус объекта недвижимости

In [20]:
df = pd.get_dummies(df, columns=['status'], dtype=int)
df.sample(5)

,property_type,baths,homefacts,fireplace,schools,sqft,beds,state,stories,target,lat,lon,pool,status_Active,status_for_sale
263211,single-family home,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,"[{'rating': ['3/10', '5/10', '4/10'], 'data': ...","1,647 sqft",3 Beds,TX,NaN,196950.0,31.782776,-106.204888,0,0,1
339250,lot/land,NaN,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",NaN,"[{'rating': ['3/10', '2/10', '2/10'], 'data': ...",768 sqft,2 Beds,TX,NaN,146999.0,32.804850,-96.803427,0,0,1
296802,Ranch,NaN,"{'atAGlanceFacts': [{'factValue': '1953', 'fac...",NaN,"[{'rating': ['2', '1', '2', '1', '1', '2', '8'...",816,NaN,OH,NaN,23000.0,41.173227,-80.706707,0,1,0
328018,single-family home,5 Baths,"{'atAGlanceFacts': [{'factValue': '2010', 'fac...",yes,"[{'rating': ['9/10', '5/10'], 'data': {'Distan...","3,665 sqft",4 Beds,CA,NaN,1456.0,33.625418,-117.661460,1,0,1
138341,Single Family,2.0,"{'atAGlanceFacts': [{'factValue': '1991', 'fac...",Not Applicable,"[{'rating': ['3/10', '3/10', '3/10'], 'data': ...","1,015",3,TX,1.0,157000.0,29.444152,-98.583305,0,0,0


In [21]:
missing_values_per_column(df)

Количество значений NaN по каждой колонке:
property_type       20316
baths               66145
homefacts               0
fireplace          160461
schools                 0
sqft                25495
beds                55949
state                   0
stories             89293
target                  0
lat                     0
lon                     0
pool                    0
status_Active           0
status_for_sale         0
dtype: int64


bath_series = df['baths'].value_counts()
bath_df = bath_series.to_frame()
bath_df.reset_index(inplace=True)
bath_df['percent'] = bath_df['count'] / df.shape[0] * 100
# bath_df[bath_df['percent'] >= 1]
bath_df
# bath_series
bath_df.to_csv('./data/bath.csv')


filtered_bath_df = bath_df.loc[bath_df['percent'] >= 0.5]
filtered_bath_df

bath_df['new_baths'] = bath_df['baths'].apply(convert_bath)
bath_df['new_baths'] = bath_df['new_baths'].fillna(0)
bath_df.to_csv('data/bath_df.csv')

In [22]:
df.sample(5, random_state=42)

,property_type,baths,homefacts,fireplace,schools,sqft,beds,state,stories,target,lat,lon,pool,status_Active,status_for_sale
279916,single-family home,2.5 Baths,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,"[{'rating': ['5/10', '6/10', '4/10'], 'data': ...","2,306 sqft",4 Beds,FL,1.0,306500.0,30.538322,-87.409632,0,0,1
109087,single-family home,3 Baths,"{'atAGlanceFacts': [{'factValue': '2008', 'fac...",NaN,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...","2,244 sqft",3 Beds,FL,NaN,250000.0,28.177260,-81.436678,0,0,1
330910,single-family home,2 Baths,"{'atAGlanceFacts': [{'factValue': '1966', 'fac...",yes,"[{'rating': ['5/10', '5/10', '5/10'], 'data': ...","1,797 sqft",3 Beds,CA,NaN,310000.0,36.826229,-119.892235,0,0,1
110312,single-family home,2 Baths,"{'atAGlanceFacts': [{'factValue': '2017', 'fac...",NaN,"[{'rating': ['8/10', '5/10', '5/10', '7/10', '...","1,943 sqft",4 Beds,FL,1.0,379900.0,25.759996,-80.218591,1,0,1
157703,Single Family Home,3,"{'atAGlanceFacts': [{'factValue': '1967', 'fac...",NaN,"[{'rating': ['3', '4', '2', '3', '6', '1', 'NR...",1268,4.0,MD,2,349900.0,38.795738,-76.916911,0,1,0


In [23]:
df['baths'] = df['baths'].astype(str)
df['new_baths'] = df['baths'].apply(convert_bath)
df['new_baths'] = df['new_baths'].fillna(0)

In [24]:
df.drop(labels=['baths'], axis=1, inplace=True)

df = df.rename(columns={
    'new_baths': 'baths',
})

In [25]:
df['property_type'] = df['property_type'].str.lower()

In [26]:
df['property_type'] = df['property_type'].astype(str)

In [27]:
pattern = r'single.*family'  # Шаблон для поиска
new_value = 'single_family'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))


In [28]:
pattern = r'mult.*family'  # Шаблон для поиска
new_value = 'multi_family'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))


In [29]:
pattern = r'land'  # Шаблон для поиска
new_value = 'land'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [30]:
pattern = r'condo'  # Шаблон для поиска
new_value = 'condo'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [31]:
pattern = r'ranch'  # Шаблон для поиска
new_value = 'ranch'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

In [32]:
pattern = r'nan'  # Шаблон для поиска
new_value = 'other'  # Новое значение для замены

df['property_type'] = df['property_type'].apply(replace_based_on_pattern, args=(pattern, new_value))

Обработаем категории с целью уменьшения списка

In [35]:
# Рассчитать частоты категорий
property_category_counts = df['property_type'].value_counts()

# Рассчитать процентное соотношение категорий
total_count = len(df)
property_category_percentages = (property_category_counts / total_count) * 100

threshold_percentage = 1

# Найти редкие категории
rare_categories = property_category_percentages[property_category_percentages < threshold_percentage].index

df['property_cleaned'] = df['property_type'].apply(lambda x: 'other' if x in rare_categories else x)

In [36]:
_df = value_percentage(df, 'property_cleaned')
_df.head(50)

,property_cleaned,count,percent
0,single_family,142388,59.663193
1,other,39072,16.371887
2,land,24250,10.161196
3,townhouse,10129,4.244237
4,multi_family,9968,4.176775
5,condo,5290,2.216607
6,traditional,4819,2.019250
7,ranch,2737,1.146853


In [ ]:
df['property_type'].value_counts()

In [ ]:
df.sample(10)

In [ ]:
missing_values_per_column(df)